We did the following process to the matches dataset:
1. Drop walkover matches.
2. Drop the non-professional matches.
3. Modify the date column so that they have the same format.
4. Create a column 'year' from tournaments dataset.
5. Sort the dataframe by year and tournaments.
6. Reverse the order the matches within a tournament since oringally they are in the reversed order of 
    stages (for example, final, semi-final, quarter-final, ...)
7. Drop the matches whose score is 0-0
Notice:
1. 2/3 of dates are missing.
2. The tournaments corresponding to the matches are roughly in the correct order. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
matches = pd.read_csv('Raw Data/Kaggle Snooker Data (1982-2020)/matches_r.csv')

In [3]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193530 entries, 0 to 193529
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   tournament_id  193530 non-null  int64 
 1   match_id       193530 non-null  int64 
 2   date           54819 non-null   object
 3   stage          193530 non-null  object
 4   best_of        193530 non-null  int64 
 5   player1_name   193530 non-null  object
 6   player1_url    193530 non-null  object
 7   player2_name   193530 non-null  object
 8   player2_url    193530 non-null  object
 9   score1         193530 non-null  int64 
 10  score2         193530 non-null  int64 
 11  frames_scores  70252 non-null   object
 12  is_walkover    193530 non-null  bool  
dtypes: bool(1), int64(5), object(7)
memory usage: 17.9+ MB


The dataset misses lots of dates and frame scores.

In [4]:
matches.sample(5)

,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover
16485,475,61609,NaN,Round 3,9,Stephen Roberts,https://cuetracker.net/players/stephen-roberts,Stuart Green,https://cuetracker.net/players/stuart-green,5,1,NaN,False
158444,1348,130734,2015-12-19,Last 64,9,Stuart Bingham,https://cuetracker.net/players/stuart-bingham,Anthony Hamilton,https://cuetracker.net/players/anthony-hamilton,5,4,"0-138(94); 136(51,81)-0; 1-94(53); 71(60)-29; ...",False
169048,2266,153357,2017-02-23,Last 128,1,Nigel Bond,https://cuetracker.net/players/nigel-bond,Mark Allen,https://cuetracker.net/players/mark-allen,1,0,46-36,False
182863,3283,203869,NaN,Group 6,5,Jörg Schneidewindt,https://cuetracker.net/players/jorg-schneidewindt,Antonio Aguado Rodriguez,https://cuetracker.net/players/antonio-aguado-...,3,1,NaN,False
145796,1297,130348,2015-04-27,Group 14,7,Ehsan Heydarinezhad,https://cuetracker.net/players/ehsan-heydarine...,Shuji Hase,https://cuetracker.net/players/shuji-hase,4,1,6-66; 97-8; 60-24; 64-33; 61-25,False


In [5]:
#Drop walkover matches
matches = matches[matches['is_walkover']==False]


In [6]:
matches['date'].tail(10)

193520    2019-10-08
193521    2019-10-07
193522    2019-10-07
193523    2019-10-07
193524    2019-10-07
193525    2019-10-08
193526    2019-10-08
193527    2019-10-07
193528    2019-10-08
193529    2019-10-08
Name: date, dtype: object

The matches are not strictly in the chronological order.

In [7]:
#Print the dates for the last tournament
print(matches[matches['tournament_id']==3275].date)

193434    2019-10-24
193435    2019-10-24
193436    2019-10-24
193437    2019-10-23
193438    2019-10-23
             ...    
193525    2019-10-08
193526    2019-10-08
193527    2019-10-07
193528    2019-10-08
193529    2019-10-08
Name: date, Length: 96, dtype: object


The dates are roughly in the backwards order for this tournament. This is because cuetracker.net presents the date in reversed order of stage, e.g. an order of 'final, semi-final, quaterfinal, last 16, last 32, last 64, last 128, prequalifying 1'.

In [8]:
#The tournaments datasets
tourns = pd.read_csv('Raw Data/Kaggle Snooker Data (1982-2020)/tournaments.csv')

In [9]:
tourns.head()

,id,season,year,name,full_name,url,status,category,prize,country,city
0,753,1982-1983,1982,UK Championship,1982 UK Championship,https://cuetracker.net/tournaments/uk-champion...,Professional,Non-ranking,47000.0,England,Preston
1,1140,1982-1983,1982,World Amateur Championship - Men,1982 World Amateur Championship - Men,https://cuetracker.net/tournaments/world-amate...,Amateur,World Event,0.0,Canada,Calgary
2,762,1982-1983,1982,Professional Players Tournament,1982 Professional Players Tournament,https://cuetracker.net/tournaments/professiona...,Professional,Ranking,31500.0,England,Birmingham
3,2586,1982-1983,1982,Pontins Autumn Open,1982 Pontins Autumn Open,https://cuetracker.net/tournaments/pontins-aut...,Pro-am,Event,0.0,Wales,Prestatyn
4,754,1982-1983,1982,International Open,1982 International Open,https://cuetracker.net/tournaments/internation...,Professional,Ranking,73500.0,England,Derby


In [10]:
tourns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2720 entries, 0 to 2719
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2720 non-null   int64  
 1   season     2705 non-null   object 
 2   year       2720 non-null   int64  
 3   name       2720 non-null   object 
 4   full_name  2720 non-null   object 
 5   url        2720 non-null   object 
 6   status     2720 non-null   object 
 7   category   2720 non-null   object 
 8   prize      2719 non-null   float64
 9   country    2690 non-null   object 
 10  city       2646 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 233.9+ KB


In [11]:
tourns['status'].value_counts()

status
Amateur         1568
Professional     949
Pro-am           203
Name: count, dtype: int64

In [12]:
tourns.set_index('id', inplace = True)

In [13]:
#Store the professional tournaments id in the list pro_tourns
pro_tourns = []
for id in tourns.index.tolist():
    if tourns.loc[id, 'status'] == 'Professional':
        pro_tourns.append(id)

In [14]:
pro_tourns[:5]

[753, 762, 754, 759, 795]

In [15]:
matches.head()

,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover
0,753,82716,NaN,Final,31,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Alex Higgins,https://cuetracker.net/players/alex-higgins,16,15,20-58; 31-90; 56-52; 26-87(67); 0-114(67); 73(...,False
1,753,82718,NaN,Semi-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Tony Meo,https://cuetracker.net/players/tony-meo,9,7,71-8; 50-71(55); 31-62; 69-30; 73-61; 34-77(52...,False
2,753,82717,NaN,Semi-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,Ray Reardon,https://cuetracker.net/players/ray-reardon,9,6,28-71; 67(50)-29; 74(74)-0; 53-79; 60-54; 112(...,False
3,753,82721,NaN,Quarter-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Steve Davis,https://cuetracker.net/players/steve-davis,9,6,"1-103; 117(60,57)-6; 5-105(60); 57-60; 79-0; 2...",False
4,753,82719,NaN,Quarter-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,John Spencer,https://cuetracker.net/players/john-spencer,9,5,69(54)-31; 103-21; 72-48; 33-82; 40-56; 71-51;...,False


In [16]:
#Drop the non-professional matches
matches['status'] = 0
for pro_tourn in pro_tourns:
    matches['status'] = 'Professional'

matches = matches[matches['status']=='Professional']

In [17]:
#Drop matches where the score is 0-0
matches['total'] = matches['score1'] + matches['score2']
matches = matches[matches['total']!=0]
matches.drop('total', axis = 1)

,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover,status
0,753,82716,NaN,Final,31,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Alex Higgins,https://cuetracker.net/players/alex-higgins,16,15,20-58; 31-90; 56-52; 26-87(67); 0-114(67); 73(...,False,Professional
1,753,82718,NaN,Semi-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Tony Meo,https://cuetracker.net/players/tony-meo,9,7,71-8; 50-71(55); 31-62; 69-30; 73-61; 34-77(52...,False,Professional
2,753,82717,NaN,Semi-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,Ray Reardon,https://cuetracker.net/players/ray-reardon,9,6,28-71; 67(50)-29; 74(74)-0; 53-79; 60-54; 112(...,False,Professional
3,753,82721,NaN,Quarter-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Steve Davis,https://cuetracker.net/players/steve-davis,9,6,"1-103; 117(60,57)-6; 5-105(60); 57-60; 79-0; 2...",False,Professional
4,753,82719,NaN,Quarter-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,John Spencer,https://cuetracker.net/players/john-spencer,9,5,69(54)-31; 103-21; 72-48; 33-82; 40-56; 71-51;...,False,Professional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193525,3275,203168,2019-10-08,Group 1,5,Neil Robertson,https://cuetracker.net/players/neil-robertson,Luca Brecel,https://cuetracker.net/players/luca-brecel,3,1,NaN,False,Professional
193526,3275,203174,2019-10-08,Group 1,5,Neil Robertson,https://cuetracker.net/players/neil-robertson,Jack Lisowski,https://cuetracker.net/players/jack-lisowski,3,1,NaN,False,Professional
193527,3275,203167,2019-10-07,Group 1,5,Mark Selby,https://cuetracker.net/players/mark-selby,Luca Brecel,https://cuetracker.net/players/luca-brecel,3,0,NaN,False,Professional
193528,3275,203169,2019-10-08,Group 1,5,Mark Selby,https://cuetracker.net/players/mark-selby,Ryan Day,https://cuetracker.net/players/ryan-day,3,2,NaN,False,Professional


In [18]:
matches.head()


,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover,status,total
0,753,82716,NaN,Final,31,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Alex Higgins,https://cuetracker.net/players/alex-higgins,16,15,20-58; 31-90; 56-52; 26-87(67); 0-114(67); 73(...,False,Professional,31
1,753,82718,NaN,Semi-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Tony Meo,https://cuetracker.net/players/tony-meo,9,7,71-8; 50-71(55); 31-62; 69-30; 73-61; 34-77(52...,False,Professional,16
2,753,82717,NaN,Semi-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,Ray Reardon,https://cuetracker.net/players/ray-reardon,9,6,28-71; 67(50)-29; 74(74)-0; 53-79; 60-54; 112(...,False,Professional,15
3,753,82721,NaN,Quarter-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Steve Davis,https://cuetracker.net/players/steve-davis,9,6,"1-103; 117(60,57)-6; 5-105(60); 57-60; 79-0; 2...",False,Professional,15
4,753,82719,NaN,Quarter-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,John Spencer,https://cuetracker.net/players/john-spencer,9,5,69(54)-31; 103-21; 72-48; 33-82; 40-56; 71-51;...,False,Professional,14


In [19]:
matches.reset_index(inplace=True)

In [20]:
#Inspect the date column.
def get_len(s):
    return len(s)
datelen = matches.loc[:,'date'].dropna().apply(get_len)
datelen.value_counts()

date
10    54632
18       96
Name: count, dtype: int64

Some dates have extra length

In [21]:
#This cell truncate the date values with extra length.
long_date_index = datelen[datelen == 18].index.tolist()
def truncate_date(s):
    return s[:10]

matches.loc[long_date_index, 'date'] = matches.loc[long_date_index, 'date'].apply(truncate_date)
matches.loc[long_date_index, 'date']


60360     2001-07-20
90254     2008-04-18
141295    2015-05-10
141296    2015-05-10
141297    2015-05-10
             ...    
162378    2017-04-25
162379    2017-04-25
162380    2017-04-25
162381    2017-04-25
173086    2018-04-21
Name: date, Length: 96, dtype: object

In [22]:
#Add a column 'year' to the dataframe
matches['year']=0
for id in tourns.index:
    indeces =matches[matches['tournament_id']==id].index
    matches.loc[indeces, 'year'] = tourns.loc[id, 'year']

In [23]:
#Create our own tournament id. We will use it to sort the dataframe.
matches['sorted_id'] = 0
j = 1
for id in tourns.index:
    rows = matches[matches['tournament_id']==id].index
    matches.loc[rows, 'sorted_id'] = j
    j+=1

In [24]:
matches.reset_index()

,level_0,index,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover,status,total,year,sorted_id
0,0,0,753,82716,NaN,Final,31,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Alex Higgins,https://cuetracker.net/players/alex-higgins,16,15,20-58; 31-90; 56-52; 26-87(67); 0-114(67); 73(...,False,Professional,31,1982,1
1,1,1,753,82718,NaN,Semi-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Tony Meo,https://cuetracker.net/players/tony-meo,9,7,71-8; 50-71(55); 31-62; 69-30; 73-61; 34-77(52...,False,Professional,16,1982,1
2,2,2,753,82717,NaN,Semi-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,Ray Reardon,https://cuetracker.net/players/ray-reardon,9,6,28-71; 67(50)-29; 74(74)-0; 53-79; 60-54; 112(...,False,Professional,15,1982,1
3,3,3,753,82721,NaN,Quarter-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Steve Davis,https://cuetracker.net/players/steve-davis,9,6,"1-103; 117(60,57)-6; 5-105(60); 57-60; 79-0; 2...",False,Professional,15,1982,1
4,4,4,753,82719,NaN,Quarter-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,John Spencer,https://cuetracker.net/players/john-spencer,9,5,69(54)-31; 103-21; 72-48; 33-82; 40-56; 71-51;...,False,Professional,14,1982,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,188837,193525,3275,203168,2019-10-08,Group 1,5,Neil Robertson,https://cuetracker.net/players/neil-robertson,Luca Brecel,https://cuetracker.net/players/luca-brecel,3,1,NaN,False,Professional,4,2020,2720
188838,188838,193526,3275,203174,2019-10-08,Group 1,5,Neil Robertson,https://cuetracker.net/players/neil-robertson,Jack Lisowski,https://cuetracker.net/players/jack-lisowski,3,1,NaN,False,Professional,4,2020,2720
188839,188839,193527,3275,203167,2019-10-07,Group 1,5,Mark Selby,https://cuetracker.net/players/mark-selby,Luca Brecel,https://cuetracker.net/players/luca-brecel,3,0,NaN,False,Professional,3,2020,2720
188840,188840,193528,3275,203169,2019-10-08,Group 1,5,Mark Selby,https://cuetracker.net/players/mark-selby,Ryan Day,https://cuetracker.net/players/ryan-day,3,2,NaN,False,Professional,5,2020,2720


In [25]:
#Revered the order of matches for each tournament. 
#(Original order is based on stages and is in the reversed order,
#for example, final, semi-final, quater-final, ...)
new_index = []
for id in tourns.index:
    rows = matches[matches['tournament_id']==id].index.tolist()
    new_index = new_index + rows[::-1]
matches['index'] = np.array(new_index)
matches.set_index('index', inplace=True)
matches.head()

,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover,status,total,year,sorted_id
index,,,,,,,,,,,,,,,,,
52,753,82716,NaN,Final,31,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Alex Higgins,https://cuetracker.net/players/alex-higgins,16,15,20-58; 31-90; 56-52; 26-87(67); 0-114(67); 73(...,False,Professional,31,1982,1
51,753,82718,NaN,Semi-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Tony Meo,https://cuetracker.net/players/tony-meo,9,7,71-8; 50-71(55); 31-62; 69-30; 73-61; 34-77(52...,False,Professional,16,1982,1
50,753,82717,NaN,Semi-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,Ray Reardon,https://cuetracker.net/players/ray-reardon,9,6,28-71; 67(50)-29; 74(74)-0; 53-79; 60-54; 112(...,False,Professional,15,1982,1
49,753,82721,NaN,Quarter-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Steve Davis,https://cuetracker.net/players/steve-davis,9,6,"1-103; 117(60,57)-6; 5-105(60); 57-60; 79-0; 2...",False,Professional,15,1982,1
48,753,82719,NaN,Quarter-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,John Spencer,https://cuetracker.net/players/john-spencer,9,5,69(54)-31; 103-21; 72-48; 33-82; 40-56; 71-51;...,False,Professional,14,1982,1


In [26]:
matches = matches.sort_values(by=['year', 'sorted_id', 'index'])
matches.reset_index()

,index,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover,status,total,year,sorted_id
0,0,753,82766,NaN,Round 1,17,Colin Roscoe,https://cuetracker.net/players/colin-roscoe,Jackie Rea,https://cuetracker.net/players/jackie-rea,9,6,34-63; 40-56; 33-68; 70(58)-47; 64-23; 94(62)-...,False,Professional,15,1982,1
1,1,753,82770,NaN,Round 1,17,Tommy Murphy,https://cuetracker.net/players/tommy-murphy,Clive Everton,https://cuetracker.net/players/clive-everton,9,4,69(59)-24; 77-34; 77-64; 58-52; 11-68; 37-62; ...,False,Professional,13,1982,1
2,2,753,82771,NaN,Round 1,17,Vic Harris,https://cuetracker.net/players/vic-harris,Marcus Owen,https://cuetracker.net/players/marcus-owen,9,4,36-40; 71(69)-43; 63-18; 14-87; 65-37; 35-92; ...,False,Professional,13,1982,1
3,3,753,82765,NaN,Round 1,17,Bob Harris,https://cuetracker.net/players/bob-harris,Graham Cripsey,https://cuetracker.net/players/graham-cripsey,9,6,78-61; 76-44; 65(59)-72; 29-90; 66-57; 88(65)-...,False,Professional,15,1982,1
4,4,753,82767,NaN,Round 1,17,Geoff Foulds,https://cuetracker.net/players/geoff-foulds,Matt Gibson,https://cuetracker.net/players/matt-gibson,9,3,30-77; 58-11; 50-76; 58-14; 42-72; 59-36; 53-1...,False,Professional,12,1982,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188837,188837,3275,203232,2019-10-23,Group 4,5,Ali Carter,https://cuetracker.net/players/ali-carter,Joe Perry,https://cuetracker.net/players/joe-perry,3,1,NaN,False,Professional,4,2020,2720
188838,188838,3275,203229,2019-10-23,Group 4,5,Ali Carter,https://cuetracker.net/players/ali-carter,Matthew Selt,https://cuetracker.net/players/matthew-selt,3,2,NaN,False,Professional,5,2020,2720
188839,188839,3275,203250,2019-10-24,Group 4 - Semi-final,5,Graeme Dott,https://cuetracker.net/players/graeme-dott,Kyren Wilson,https://cuetracker.net/players/kyren-wilson,3,2,NaN,False,Professional,5,2020,2720
188840,188840,3275,203249,2019-10-24,Group 4 - Semi-final,5,Scott Donaldson,https://cuetracker.net/players/scott-donaldson,Joe Perry,https://cuetracker.net/players/joe-perry,3,2,NaN,False,Professional,5,2020,2720


In [27]:
matches.to_csv('matches_v1.csv')